In [4]:
import numpy as np
import random, time, torch
import torch.nn.init as init
import tensorflow as tf
# 設置隨機種子
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
tf.random.set_seed(seed)

t1 = time.monotonic(); startime = time.ctime(); startime = startime.split()
print("end time: ", startime[1:4])

end time:  ['Jul', '22', '14:12:00']


In [20]:
# 生成输入数据
nb = 32
input_tf = tf.random.normal((nb, 10, 1))
targ_tf = tf.random.normal((nb, 1))

t1 = time.monotonic(); startime = time.ctime(); startime = startime.split()
print("end time: ", startime[1:4])

end time:  ['Jul', '22', '14:29:24']


In [21]:
'''Tensorflow'''
from tensorflow.keras import layers, models, optimizers, initializers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform, HeUniform

m_tf = models.Sequential([
    tf.keras.layers.Conv1D(3, 3, activation='relu', use_bias=True, kernel_initializer=GlorotUniform(seed=seed)),
    tf.keras.layers.Conv1D(3, 3, activation='relu', use_bias=True, kernel_initializer=GlorotUniform(seed=seed)),
    layers.Flatten(),
    layers.Dense(6, activation='relu', kernel_initializer=GlorotUniform(seed=seed)),
    layers.Dense(1, kernel_initializer=GlorotUniform(seed=seed))])     #

ouput_tf = m_tf(input_tf)          # 檢查output數據，用於跟pytorch比較
print('ouput_tf = \n', [ouput_tf.shape, ouput_tf])

ouput_tf = 
 [TensorShape([32, 1]), <tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.31212005],
       [0.36599502],
       [0.3744991 ],
       [0.26357841],
       [0.4316152 ],
       [0.21662524],
       [0.71965253],
       [0.09666546],
       [0.3032107 ],
       [0.23645878],
       [0.07455332],
       [0.3313919 ],
       [0.40289629],
       [0.37302992],
       [0.6448723 ],
       [0.32537276],
       [0.38375035],
       [0.18190469],
       [0.24065128],
       [0.499044  ],
       [0.52254426],
       [0.2628636 ],
       [0.57755244],
       [0.9148287 ],
       [0.39798233],
       [0.46651536],
       [0.49470246],
       [0.31223896],
       [0.5721373 ],
       [0.8103552 ],
       [0.5558058 ],
       [0.81349385]], dtype=float32)>]


In [22]:
'''Pytorch'''
import torch.nn.init as init
from torch import nn, optim

input_py = torch.from_numpy(input_tf.numpy().reshape(nb, 1, 10)).float()     # X_train_tf
targ_py = torch.from_numpy(targ_tf.numpy().reshape(nb, 1)).float()    # y_train_tf.reshape(5, 1)

class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 3, 3)
        self.conv2 = nn.Conv1d(3, 3, 3)
        self.flatt = nn.Flatten()
        self.fc1 = nn.Linear(18, 6)
        self.fc2 = nn.Linear(6, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        xs = x.permute(0, 2, 1)            # 扭轉命運的一行
        xs = self.flatt(xs)
        xs = torch.relu(self.fc1(xs))
        xs = self.fc2(xs)
        return xs

m_py = PyTorchModel()

'''將tensorflow的超參數轉移到pytorch上面'''
m_tf_weight = m_tf.get_weights()
m_py_weights_pre = [np.array(w) for w in m_tf_weight]
with torch.no_grad():
    m_py.conv1.weight.data = nn.Parameter(torch.tensor(m_py_weights_pre[0].T, dtype=torch.float32))
    m_py.conv1.bias.data = nn.Parameter(torch.tensor(m_py_weights_pre[1].T, dtype=torch.float32))
    m_py.conv2.weight.data = nn.Parameter(torch.tensor(m_py_weights_pre[2].T, dtype=torch.float32))
    m_py.conv2.bias.data = nn.Parameter(torch.tensor(m_py_weights_pre[3].T, dtype=torch.float32))
    m_py.fc1.weight.data = nn.Parameter(torch.tensor(m_py_weights_pre[4].T, dtype=torch.float32).contiguous())
    m_py.fc1.bias.data = nn.Parameter(torch.tensor(m_py_weights_pre[5].T, dtype=torch.float32))
    m_py.fc2.weight.data = nn.Parameter(torch.tensor(m_py_weights_pre[6].T, dtype=torch.float32).contiguous())
    m_py.fc2.bias.data = nn.Parameter(torch.tensor(m_py_weights_pre[7].T, dtype=torch.float32))

ouput_py = m_py(input_py)          # 檢查output數據，確認tensorflow model的權重友被遷移到pytorch
print('ouput_py = \n', [ouput_py])

ouput_py = 
 [tensor([[0.3121],
        [0.3660],
        [0.3745],
        [0.2636],
        [0.4316],
        [0.2166],
        [0.7197],
        [0.0967],
        [0.3032],
        [0.2365],
        [0.0746],
        [0.3314],
        [0.4029],
        [0.3730],
        [0.6449],
        [0.3254],
        [0.3838],
        [0.1819],
        [0.2407],
        [0.4990],
        [0.5225],
        [0.2629],
        [0.5776],
        [0.9148],
        [0.3980],
        [0.4665],
        [0.4947],
        [0.3122],
        [0.5721],
        [0.8104],
        [0.5558],
        [0.8135]], grad_fn=<AddmmBackward0>)]


In [23]:
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

'''Tensorflow'''
t1 = time.monotonic(); startime = time.ctime(); startime = startime.split()
print("start time: ", startime[1:4])

dec_rate, dec_step = 0.5, 10     
opti_tf = tf.keras.optimizers.Adam(0.001)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor = dec_rate, patience = dec_step, min_lr=1e-5)
m_tf.compile(loss='mse', optimizer=opti_tf, metrics=['mae'])

history = m_tf.fit(input_tf, targ_tf, epochs=5, verbose=1, callbacks=[reduce_lr])    # X_train_tf, y_train_tf

'''Pytorch'''
t1 = time.monotonic(); startime = time.ctime(); startime = startime.split()
print("\nstart time: ", startime[1:4])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()  
opti_py = optim.Adam(m_py.parameters(), lr=0.001)
reduce_rate = optim.lr_scheduler.ReduceLROnPlateau(opti_py, mode='min', factor = dec_rate, patience = dec_step, verbose=True)

traloss, valioss = [], []
for t in range(5):
  #  T r a i n
    m_py.train()
    y_pred = m_py(input_py)         # input_py
    loss = criterion(y_pred, targ_py)   # X_train, y_pred, y_train
    opti_py.zero_grad()
    loss.backward()
    opti_py.step()
    reduce_rate.step(loss)
    traloss.append(np.round(torch.sqrt(loss).item(), 5))
    print(f'loss_py of {t+1} epoches = ', '%.4f'%float(loss.detach().numpy()))
    
t1 = time.monotonic(); startime = time.ctime(); startime = startime.split()
print("\nend time: ", startime[1:4])

start time:  ['Jul', '22', '14:29:27']
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.4581 - mae: 1.0440 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1.4414 - mae: 1.0383 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.4254 - mae: 1.0327 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.4101 - mae: 1.0270 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.3954 - mae: 1.0214 - learning_rate: 0.0010

start time:  ['Jul', '22', '14:29:29']
loss_py of 1 epoches =  1.4581
loss_py of 2 epoches =  1.4414
loss_py of 3 epoches =  1.4254
loss_py of 4 epoches =  1.4101
loss_py of 5 epoches =  1.3954

end time:  ['Jul', '22', '14:29:29']
